In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-27 03:28:18.500626: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 03:28:18.503647: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 03:28:18.535924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 03:28:18.535976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 03:28:18.537648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test.values.reshape(-1), y_pred_classes, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    accuracy = accuracy_score(y_test, y_pred_classes)

    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='macro')
    
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='weighted')
    
    auc = metrics.roc_auc_score(y_test.values.reshape(-1), y_pred_classes, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")


In [6]:
x_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_train.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_test.csv', low_memory=False)

In [7]:
model = joblib.load("/home/jovyan/UNSW/dl/new_dl/dnn.joblib")

2024-02-27 03:28:33.338847: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://06e55477b9304a2cb71db4aea3044a63: INVALID_ARGUMENT: ram://06e55477b9304a2cb71db4aea3044a63 is a directory.


In [8]:
x_test_trimmed = x_test[100000:104000]
y_test_trimmed = y_test[100000:104000]

In [9]:
x_test.shape

(175341, 56)

In [10]:
np.unique(y_test_trimmed)

array([0, 1])

In [11]:
y_pred = model.predict(x_test)

5480/5480 [==============================] - 3s 560us/step


In [12]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=(56,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [13]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.01, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_37779/1631696788.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA:   0%|          | 0/4000 [00:00<?, ?it/s]

JSMA: 100%|██████████| 4000/4000 [00:17<00:00, 225.25it/s]


In [14]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 901us/step
Accuracy: 0.81425
AUC: 0.831695323230421

macro
Precision: 0.7433674462445699
Recall: 0.8316953232304207
F1 Score: 0.7643274741115078

weighted
Precision: 0.8693096005792558
Recall: 0.81425
F1 Score: 0.8282152910551729

Mean FNR: 0.16830467676957916
Mean TNR: 0.8316953232304207
Mean FPR: 0.16830467676957916
Mean TPR: 0.8316953232304207


In [15]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.05, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_37779/2229238815.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:17<00:00, 226.55it/s]


In [16]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 626us/step
Accuracy: 0.8085
AUC: 0.8244691683548238

macro
Precision: 0.7374497991967872
Recall: 0.8244691683548238
F1 Score: 0.7575267219163871

weighted
Precision: 0.8650368975903614
Recall: 0.8085
F1 Score: 0.8230082067391743

Mean FNR: 0.17553083164517616
Mean TNR: 0.8244691683548238
Mean FPR: 0.17553083164517616
Mean TPR: 0.8244691683548238


In [37]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.1, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_53013/1183469131.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA:   0%|          | 0/4000 [00:00<?, ?it/s]

JSMA: 100%|██████████| 4000/4000 [00:24<00:00, 163.48it/s]


In [51]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 597us/step
Accuracy: 0.805
AUC: 0.8177571166734805

macro
Precision: 0.733054145202535
Recall: 0.8177571166734805
F1 Score: 0.7526636225266363

weighted
Precision: 0.861078642107994
Recall: 0.805
F1 Score: 0.8196769406392695

Mean FNR: 0.18224288332651944
Mean TNR: 0.8177571166734805
Mean FPR: 0.18224288332651944
Mean TPR: 0.8177571166734805


In [52]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.15, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_53013/1855246553.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:24<00:00, 164.99it/s]


In [53]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 575us/step
Accuracy: 0.80875
AUC: 0.8110982744813414

macro
Precision: 0.7331347530100453
Recall: 0.8110982744813415
F1 Score: 0.7534042900400718

weighted
Precision: 0.8573673345360796
Recall: 0.80875
F1 Score: 0.8222140487026527

Mean FNR: 0.18890172551865853
Mean TNR: 0.8110982744813415
Mean FPR: 0.18890172551865853
Mean TPR: 0.8110982744813415


In [54]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.2, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_53013/1114785325.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:24<00:00, 164.93it/s]


In [55]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 576us/step
Accuracy: 0.80775
AUC: 0.8091161253079643

macro
Precision: 0.7318403555056374
Recall: 0.8091161253079642
F1 Score: 0.751963307986983

weighted
Precision: 0.8562097035089185
Recall: 0.80775
F1 Score: 0.821248203286228

Mean FNR: 0.19088387469203572
Mean TNR: 0.8091161253079642
Mean FPR: 0.19088387469203572
Mean TPR: 0.8091161253079642


In [56]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.25, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_53013/2667259709.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:24<00:00, 164.07it/s]


In [57]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 575us/step
Accuracy: 0.80775
AUC: 0.8023520125436585

macro
Precision: 0.7296711921114254
Recall: 0.8023520125436585
F1 Score: 0.7496474024575086

weighted
Precision: 0.8524707057198911
Recall: 0.80775
F1 Score: 0.8206851397746386

Mean FNR: 0.19764798745634143
Mean TNR: 0.8023520125436585
Mean FPR: 0.19764798745634143
Mean TPR: 0.8023520125436585


In [58]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_53013/1503088926.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:24<00:00, 163.15it/s]


In [59]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 589us/step
Accuracy: 0.80925
AUC: 0.7983356531139418

macro
Precision: 0.7294713819951353
Recall: 0.7983356531139418
F1 Score: 0.7493836208400821

weighted
Precision: 0.8504227488030255
Recall: 0.80925
F1 Score: 0.8215294977202696

Mean FNR: 0.2016643468860582
Mean TNR: 0.7983356531139418
Mean FPR: 0.2016643468860582
Mean TPR: 0.7983356531139418
